In [1]:
pip install transformers torch datasets evaluate

Note: you may need to restart the kernel to use updated packages.


In [9]:
!pip install scikit-learn==0.24

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('labeled_reviews.csv')

C:\Users\Victor\AppData\Local\Temp\ipykernel_10588\4285109436.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('labeled_reviews.csv')


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
import evaluate

In [4]:
df = df.dropna()
df = df[['Review', 'isSuggestion']]
df

,Review,isSuggestion
0,better than any you can get at a restaurant!,False
1,"I cut back on the mayo, and made up the differ...",True
2,i think i did something wrong because i could ...,False
3,easily the best i have ever had. juicy flavor...,False
4,An excellent dish.,False
...,...,...
1003,I doubled the amount of coffee powder in this ...,True
1004,I love this dish. To me it is a winter comfort...,False
1005,We really enjoyed this chicken! I added the or...,True
1006,Sounds good !,False


In [5]:
dataset = Dataset.from_pandas(df).train_test_split(test_size=0.2)
dataset = dataset.remove_columns(["__index_level_0__"])
dataset = dataset.rename_column("Review", "text")
dataset = dataset.rename_column("isSuggestion", "label")
print(dataset['train'][1])

{'text': 'tasty. you can substitute granulated sugar for the palm sugar.', 'label': True}


In [6]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)
#tokenized_datasets['train'][1]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [10]:
## CPU training
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)


metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    print(logits, labels)
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


training_args = TrainingArguments(output_dir="trainer", evaluation_strategy="epoch")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [7]:
## GPU training
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2).to("cuda")
#model = AutoModelForSequenceClassification.from_pretrained("./test_model").to("cuda")


metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    print(logits, labels)
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


training_args = TrainingArguments(per_device_train_batch_size=4, gradient_accumulation_steps=4,
                                  num_train_epochs=5, output_dir="trainer", evaluation_strategy="epoch")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [8]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\Victor\miniconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 806
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 4
  Total optimization steps = 250
  Number of trainable parameters = 108311810


Epoch,Training Loss,Validation Loss,Accuracy
0,No log,0.381801,0.846535
1,No log,0.363781,0.851485
2,No log,0.483966,0.866337
3,No log,0.663138,0.866337
4,No log,0.634039,0.876238


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 202
  Batch size = 8


[[-1.1237105   1.4541268 ]
 [-1.0190567   1.2225975 ]
 [-0.71004295  0.86320883]
 [ 0.9851582  -1.414731  ]
 [ 0.455781   -0.51667595]
 [ 1.5259453  -1.992947  ]
 [-0.56560564  0.9540572 ]
 [ 1.5718825  -1.9528058 ]
 [-1.0523838   1.4380155 ]
 [ 1.5324883  -1.9126877 ]
 [-0.9930875   1.202351  ]
 [-1.1659946   1.446707  ]
 [ 1.5032706  -1.9590857 ]
 [ 1.4735982  -1.984615  ]
 [ 1.6179903  -2.1259592 ]
 [ 1.4282984  -1.8949379 ]
 [-1.1716237   1.4492362 ]
 [-1.0003037   1.3582871 ]
 [-1.1582823   1.4526454 ]
 [-1.1542687   1.4452809 ]
 [ 1.5650826  -2.085507  ]
 [ 1.4244918  -1.9197546 ]
 [-1.1300865   1.4867948 ]
 [ 1.5039482  -2.0335443 ]
 [-1.101792    1.3937261 ]
 [ 1.4492002  -1.8351122 ]
 [-1.0568866   1.3034099 ]
 [ 1.4677248  -1.8842996 ]
 [ 1.517832   -1.982825  ]
 [-0.8785474   1.0219867 ]
 [ 1.5611639  -1.9763621 ]
 [-0.9093592   1.1980397 ]
 [ 1.4967957  -1.9888787 ]
 [ 1.5506704  -2.0249147 ]
 [-0.76973826  1.1444755 ]
 [ 0.29262325 -0.18039347]
 [ 1.5661187  -2.1327095 ]
 

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 202
  Batch size = 8


...

[[-2.83636     3.3247542 ]
 [-2.1598759   2.3556461 ]
 [ 2.1138601  -2.6696517 ]
 [ 2.8243814  -3.7703583 ]
 [ 2.272965   -3.202106  ]
 [ 2.887612   -3.702949  ]
 [ 2.0069776  -2.8580718 ]
 [ 2.992893   -3.7730908 ]
 [-2.9262838   3.4135096 ]
 [ 2.8871074  -3.635265  ]
 [-2.768259    3.2271204 ]
 [-2.904539    3.3678262 ]
 [ 2.8573852  -3.6490169 ]
 [ 2.7936554  -3.6868663 ]
 [ 2.960144   -3.6898906 ]
 [ 2.9218311  -3.7078686 ]
 [-2.435907    2.812121  ]
 [-2.6366653   3.1372132 ]
 [-2.8599927   3.3304803 ]
 [-2.8692257   3.3573768 ]
 [ 2.8690453  -3.7299516 ]
 [ 2.8213553  -3.7405763 ]
 [-2.8264372   3.3178298 ]
 [ 2.8668072  -3.7159328 ]
 [-2.8133078   3.2799838 ]
 [ 2.9375453  -3.5974903 ]
 [-0.11200872  0.09063104]
 [ 2.829506   -3.5754523 ]
 [ 2.894609   -3.7285964 ]
 [-2.4799337   2.7775655 ]
 [ 3.0377786  -3.852217  ]
 [-2.3413484   2.5900156 ]
 [ 2.7973452  -3.5888283 ]
 [ 2.9028432  -3.7812536 ]
 [-2.2815635   2.6611018 ]
 [ 0.5705215  -0.77083844]
 [ 2.8981793  -3.6824086 ]
 

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 202
  Batch size = 8


[[-2.9569836   3.433156  ]
 [ 1.4925021  -2.0173893 ]
 [ 2.5068002  -3.1438527 ]
 [ 2.939036   -3.868258  ]
 [ 2.5401945  -3.4563823 ]
 [ 2.9467452  -3.7504773 ]
 [ 2.426891   -3.3352234 ]
 [ 3.0553248  -3.8330207 ]
 [-3.0389926   3.5138304 ]
 [ 2.9618046  -3.6838892 ]
 [-2.849632    3.3102965 ]
 [-3.0182953   3.4662952 ]
 [ 2.9490201  -3.713091  ]
 [ 2.9183352  -3.7758462 ]
 [ 3.0065088  -3.7101817 ]
 [ 2.9689648  -3.742098  ]
 [-2.016796    2.388553  ]
 [-2.7267458   3.224454  ]
 [-2.9611707   3.4212492 ]
 [-2.9746606   3.4562738 ]
 [ 2.9370956  -3.7803295 ]
 [ 2.9348578  -3.837602  ]
 [-2.90588     3.3814971 ]
 [ 2.9152508  -3.7558181 ]
 [-2.850736    3.2931287 ]
 [ 2.9983246  -3.6613123 ]
 [ 1.2473527  -1.7026558 ]
 [ 2.9560819  -3.686551  ]
 [ 3.005128   -3.7977507 ]
 [-2.5124245   2.7501137 ]
 [ 3.1026142  -3.9030282 ]
 [-1.4140091   1.7320321 ]
 [ 2.950342   -3.6886754 ]
 [ 2.975609   -3.8495214 ]
 [-2.0997744   2.4873116 ]
 [ 1.4429731  -1.8844248 ]
 [ 2.9543517  -3.7113712 ]
 



Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=250, training_loss=0.16223356628417968, metrics={'train_runtime': 176.8386, 'train_samples_per_second': 22.789, 'train_steps_per_second': 1.414, 'total_flos': 1058758886768640.0, 'train_loss': 0.16223356628417968, 'epoch': 4.99})

In [52]:
trainer.save_model("test_model")

Saving model checkpoint to test_model
Configuration saved in test_model\config.json
Model weights saved in test_model\pytorch_model.bin


In [79]:
from transformers import pipeline

#model = AutoModelForSequenceClassification.from_pretrained("./test_model")

pipe = pipeline(task="sentiment-analysis", model=model, tokenizer=tokenizer, device=0)
pipe(["I put the chicken in the oven for longer.", "I substituted the pears for apples.", "I added extra paprika.", "Eggs should be added."])

[{'label': 'LABEL_1', 'score': 0.9967137575149536},
 {'label': 'LABEL_1', 'score': 0.999906063079834},
 {'label': 'LABEL_1', 'score': 0.9998805522918701},
 {'label': 'LABEL_0', 'score': 0.986866295337677}]